# Goal 

**Inputs:**

1. Take an input context text 
2. Take input sentence

**Backend:**

1. Convert input(1) to tokens 
2. wikpedia_topic_mode(tokens) -> likely_topics
3. wikpedia_topic_mode(likely_topics) -> pun_possible_words
4. pun.insert_pun( input(2), pun_possible_words ) -> new sentence

**Output:**

1. Print new sentence



In [1]:
# imports 
import pun
import pickle
from gensim.models.ldamodel import LdaModel as Lda
from gensim import corpora
from gensim.models import doc2vec
from gensim.utils import simple_preprocess

In [2]:
import pandas as pd

In [15]:
output = pun.generate_possible_pun_substitutions("I love my dog and think that she's a very sweet and caring person.",
                                                 'This is just a test')
output[1]

/home/wmvoje/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[['love',
  'doc2vec',
  1,
  0.0726928659579519,
  0.3632044792175293,
  0.2001430877575023],
 ['make',
  'doc2vec',
  1,
  0.030784627488728136,
  0.3632044792175293,
  0.08475839162294775],
 ['move',
  'doc2vec',
  1,
  0.055041541776322055,
  0.3632044792175293,
  0.15154422625761932],
 ['include',
  'doc2vec',
  1,
  0.04130730505567569,
  0.3632044792175293,
  0.11373016418923634],
 ['music',
  'doc2vec',
  1,
  0.05459013115835901,
  0.3632044792175293,
  0.1503013709411443],
 ['soul',
  'doc2vec',
  1,
  0.20526952865968637,
  0.3632044792175293,
  0.5651624371536095],
 ['die',
  'doc2vec',
  1,
  0.043342488171534826,
  0.3632044792175293,
  0.11933357282627641],
 ['bear',
  'doc2vec',
  1,
  0.046323525499532496,
  0.3632044792175293,
  0.12754117349909816],
 ['singer',
  'doc2vec',
  1,
  0.07031166939533748,
  0.3632044792175293,
  0.19358701067457548],
 ['songwriter',
  'doc2vec',
  1,
  0.09109644738676159,
  0.3632044792175293,
  0.25081311657558714],
 ['song',
  'doc2ve

In [11]:
test = pun.sentence_to_doc2vec("This is a short sentence",
                        pun.wiki_doc2vec)

In [12]:
for i in test:
    print(i)

([('regions', 0.43092002529087925), ('department', 0.3812470718185651), ('region', 0.34970959120588385), ('saint', 0.40115568200282054), ('commune', 0.41759641521331237), ('departments', 0.4597272633306995)], 0.6495630741119385, 56293)
([('regions', 0.43092002529087925), ('department', 0.3812470718185651), ('region', 0.34970959120588385), ('saint', 0.40115568200282054), ('commune', 0.41759641521331237), ('departments', 0.4597272633306995)], 0.5927094221115112, 56281)
([('regions', 0.3915072128851519), ('department', 0.346377447665742), ('region', 0.3177244484746374), ('saint', 0.364465176311895), ('martin', 0.4178022544953132), ('commune', 0.37940220698871985), ('departments', 0.41767968298160685)], 0.59075927734375, 55378)
([('regions', 0.3915072128851519), ('department', 0.346377447665742), ('region', 0.3177244484746374), ('saint', 0.364465176311895), ('martin', 0.4178022544953132), ('commune', 0.37940220698871985), ('departments', 0.41767968298160685)], 0.5778038501739502, 56302)
([

/home/wmvoje/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [10]:
pun.wiki_doc2vec_corpus[99662]

['sasamuta',
 'was',
 'the',
 'chief',
 'shinto',
 'shrine',
 'of',
 'the',
 'old',
 'bungo',
 'province',
 'it',
 'serves',
 'today',
 'as',
 'one',
 'of',
 'the',
 'ichinomiya',
 'of',
 'oita',
 'prefecture',
 'nationwide',
 'list',
 'of',
 'ichinomiya',
 'retrieved',
 'this',
 'place',
 'is',
 'special',
 'to',
 'the',
 'kami',
 'named']

In [9]:
pd.DataFrame(data=output[1])

,0,1,2,3,4,5
0,group,doc2vec,1,0.379626,0.379626,1.000000
1,people,doc2vec,2,0.063716,0.371136,0.171679
2,live,doc2vec,2,0.092359,0.371136,0.248855
3,population,doc2vec,2,0.131130,0.371136,0.353320
4,province,doc2vec,2,0.139686,0.371136,0.376373
5,municipality,doc2vec,2,0.156651,0.371136,0.422086
6,commune,doc2vec,2,0.153784,0.371136,0.414360
7,par,doc2vec,2,0.200638,0.371136,0.540606
8,head,doc2vec,5,0.064088,0.367504,0.174386
9,time,doc2vec,5,0.034093,0.367504,0.092770


In [43]:
print(len(output[1]))

81


In [13]:
def convert_topic_words_to_print(topic_words):
    output = [[]]
    topic_number = 1
    for topic in topic_words:
        if topic[1] == 'topicModel':
            return output
        if topic[2] != topic_number:
            topic_number = topic[2]
            output.append([topic[0]])
        output[topic_number-1].append(topic[0])

    return output

In [5]:
test = output[1]


test.sort(key=lambda x: x[3])

In [6]:
test.reverse()

In [7]:
test

[['cat',
  'doc2vec',
  1,
  0.3732185934559639,
  0.41739949584007263,
  0.8941519986860821],
 ['ferret',
  'doc2vec',
  3,
  0.32527786657376184,
  0.3852998614311218,
  0.8442200455654982],
 ['catfish',
  'doc2vec',
  2,
  0.31355886929641774,
  0.39072367548942566,
  0.8025079844564058],
 ['formulas',
  'doc2vec',
  4,
  0.17813471769931302,
  0.3824636936187744,
  0.46575588917695043],
 ['hunt',
  'doc2vec',
  3,
  0.12668919316311963,
  0.3852998614311218,
  0.3288067446807718],
 ['barbel',
  'doc2vec',
  2,
  0.1158402332885402,
  0.39072367548942566,
  0.29647610461136553],
 ['whisker',
  'doc2vec',
  2,
  0.09023460350423076,
  0.39072367548942566,
  0.23094224682239103],
 ['diverse',
  'doc2vec',
  2,
  0.07891399331449649,
  0.39072367548942566,
  0.20196880369649414],
 ['aquarium',
  'doc2vec',
  2,
  0.07627673721146928,
  0.39072367548942566,
  0.19521913310199088],
 ['can',
  'doc2vec',
  4,
  0.074773843566463,
  0.3824636936187744,
  0.1955057298614984],
 ['felines',
 

In [5]:
convert_topic_words_to_print(output[1])

[['call',
  'sex',
  'keep',
  'breed',
  'remove',
  'organs',
  'cat',
  'leopard',
  'pet',
  'cheetah',
  'felines',
  'furry',
  'hairless',
  'jaguar',
  'lynx',
  'puma',
  'pussycat',
  'tame',
  'wildcat'],
 ['look',
  'look',
  'group',
  'order',
  'farm',
  'food',
  'grow',
  'families',
  'diverse',
  'aquarium',
  'whisker',
  'barbel',
  'catfish'],
 ['hunt',
  'hunt',
  'body',
  'hole',
  'domesticate',
  'thin',
  'rodent',
  'rabbit',
  'ferret'],
 ['come',
  'come',
  'weight',
  'flavor',
  'vegetables',
  'meat',
  'feed',
  'salmon',
  'owners',
  'milk',
  'formulas',
  'manufacture',
  'bag',
  'can',
  'nutrient',
  'essence',
  'moist',
  'diet',
  'kitten'],
 ['people',
  'people',
  'ancient',
  'scrap',
  'foods',
  'taste',
  'today',
  'rat',
  'compare',
  'ancestors',
  'gods',
  'kinds',
  'evidence',
  'past',
  'insects',
  'ability',
  'mice',
  'eternity',
  'mummification',
  'mummify'],
 ['color',
  'color',
  'pattern',
  'shape',
  'eye',
  '

In [9]:
standard_output = output[1]

In [25]:
len(list_of_words)

150

In [26]:
list_of_words

['call',
 'sex',
 'keep',
 'breed',
 'remove',
 'organs',
 'cat',
 'leopard',
 'pet',
 'cheetah',
 'felines',
 'furry',
 'hairless',
 'jaguar',
 'lynx',
 'puma',
 'pussycat',
 'tame',
 'wildcat',
 'come',
 'weight',
 'flavor',
 'food',
 'vegetables',
 'meat',
 'feed',
 'salmon',
 'owners',
 'milk',
 'formulas',
 'manufacture',
 'bag',
 'can',
 'nutrient',
 'essence',
 'moist',
 'diet',
 'kitten',
 'color',
 'pattern',
 'shape',
 'eye',
 'chocolate',
 'prefer',
 'coat',
 'almond',
 'apricot',
 'green',
 'fur',
 'cream',
 'cinnamon',
 'lilac',
 'fawn',
 'tortoiseshell',
 'caramel',
 'tabby',
 'longhairs',
 'hunt',
 'body',
 'hole',
 'domesticate',
 'thin',
 'rodent',
 'rabbit',
 'ferret',
 'things',
 'use',
 'thing',
 'ones',
 'spell',
 'experts',
 'hello',
 'adult',
 'attention',
 'noise',
 'annoy',
 'meow',
 'create',
 'municipalities',
 'publish',
 'office',
 'access',
 'municipality',
 'merge',
 'fall',
 'mark',
 'sense',
 'protect',
 'animals',
 'live',
 'list',
 'learn',
 'eat',
 '

In [2]:
# Load the model

wiki_topicmodel = Lda.load('models/180925_wikipedia_model.individually_binned.200.gensim.')

# loading the stemmed_dict

with open('180922_stemmed_dict.p', 'rb') as tounpick:
    stemmed_dict = pickle.load(tounpick)

# Loading the doc2vec

wiki_doc2vec = doc2vec.Doc2Vec.load('models/simple_wiki_chunked_doc2vec_300_vector_10_min_word')

# loading the doc2vec corpus 

with open('models/simple_wiki_chunked_corpus_10_count_cutoff.p', 'rb') as tounpcik:
    wiki_doc2vec_corpus = pickle.load(tounpcik)

# Dealing with words from topic model

In [5]:
def get_words_from_top_topics(topic_list, model, min_topic_prob=0.1, min_word_prob=0.05):
    """
    First finding all of the words
    """
    
    list_of_words = []
    
    topic_list.sort(key=lambda tup: tup[1], reverse=True)
    
    for topic, topic_prob in topic_list:
        
        if topic_prob < min_topic_prob:
            break
        
        for word_id, word_prob in model.get_topic_terms(topic, 100):
            if word_prob < min_word_prob:
                break
            list_of_words.append(model.id2word[word_id])
    
    return list_of_words

In [6]:
def get_words_from_top_topic(topic_list, model, min_word_prob=0.05):
    """
    First finding all of the words
    """
    
    list_of_words = []
    topic_list.sort(key=lambda tup: tup[1], reverse=True)
    
    topic, topic_prob = topic_list[0]
        
    for word_id, word_prob in model.get_topic_terms(topic, 100):
        if word_prob < min_word_prob:
            break
        if model.id2word[word_id] in stemmed_dict:
            for word in stemmed_dict[model.id2word[word_id]]:
                list_of_words.append(word)
        else:
            list_of_words.append(model.id2word[word_id])

    
    return list_of_words, topic_prob

In [7]:
# Define the context

context = 'purr'
context = "Climbing is the activity of using one's hands, feet, or any other part of the body to ascend a steep object. It is done for locomotion, recreation and competition, in trades that rely on it, and in emergency rescue and military operations. It is done indoors and out, on natural and man-made structures."
# context = "The domestic cat (Felis silvestris catus or Felis catus)[1][4] is a small, typically furry, carnivorous mammal. They are often called house cats[5] when kept as indoor pets or simply cats when there is no need to distinguish them from other felids and felines. They are often valued by humans for companionship and for their ability to hunt vermin. There are more than seventy cat breeds recognized by various cat registries."

# Tokenize context 

context_tokens = pun.tokenize(context)

# Convert to bow

bag_of_words = wiki_topicmodel.id2word.doc2bow(context_tokens)

In [8]:
def sentence_to_topicmodel_words(sentence, model):
    context_tokens = pun.tokenize(sentence)
    bag_of_words = model.id2word.doc2bow(context_tokens)
    document_topics = model.get_document_topics(bag_of_words) 
    return get_words_from_top_topic(document_topics, model)

In [9]:
sentence_to_topicmodel_words(context, wiki_topicmodel)

(['wear', 'feet', 'walk'], 0.12638511)

# Dealing with doc2vec information

In [10]:
def sentence_to_doc2vec(text, model):
    """
    Iterator which spits out words that are found in 
    the most 'topical' textual elements
    """
    # parse the sentence 
    text = simple_preprocess(text)
    # Find the respective doc2vec vector
    text_vector = model.infer_vector(text)
    # find the most similar text pieces
    most_similar_documents_with_score = model.docvecs.most_similar([text_vector])
    
    for document_id, cosine_sim_score in most_similar_documents_with_score:
        
        yield (set(pun.tokenize(wiki_doc2vec_corpus[document_id], stem=False, initial_word_split=False)), cosine_sim_score)
        
    

In [11]:
output = sentence_to_doc2vec("Climbing is the activity of using one's hands, feet, or any other part of the body to ascend a steep object. It is done for locomotion, recreation and competition, in trades that rely on it, and in emergency rescue and military operations. It is done indoors and out, on natural and man-made structures.",
                             wiki_doc2vec)

In [15]:
output

<generator object sentence_to_doc2vec at 0x7f62a6ff61a8>

In [12]:
next(output)

/home/wmvoje/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


({'examples',
  'harm',
  'include',
  'inhibition',
  'organism',
  'plant',
  'relationship',
  'secretions',
  'shade',
  'taller',
  'wider'},
 0.36731237173080444)

# Builiding how it is all connected

In [13]:
def generate_possible_pun_substitutions(context, input_sentence, w2v_number=4):
    """
    Takes context and input sentence 
    
    returns list of possible substitutions with scores and the topic 
    words consdiered

    """
    
    # First process context
    doc2vec_word_generator = sentence_to_doc2vec(context, wiki_doc2vec)
    topic_words, topic_score = sentence_to_topicmodel_words(context, wiki_topicmodel)

    # Then try to generate sentences using these metrics
    output = []
    topic_words_considered = []
    
    # consider word2vec words
    for i in range(w2v_number):
        words, w2v_score = next(doc2vec_word_generator)

        topic_words_considered.extend([[word, 'doc2vec', i+1, w2v_score] for word in words])

        sub_tuples = pun.enumerate_PD_pun_subs(input_sentence, words)
        # word, sub_index, phonetic_distance

        for word, sub_index, phon_dist in sub_tuples:
            output.append([word, sub_index, phon_dist, 'doc2vec', i+1, w2v_score, phon_dist/w2v_score])

    # Now do topic words
    sub_tuples = pun.enumerate_PD_pun_subs(input_sentence, topic_words)
    for word, sub_index, phon_dist in sub_tuples:

        topic_words_considered.append([word, 'topicModel', 1, topic_score])

        output.append([word, sub_index, phon_dist, 'topicModel', 1, topic_score, phon_dist/topic_score])

    output.sort(key=lambda x: x[6])
    
    return output, topic_words_considered


In [14]:
# context = "Climbing is the activity of using one's hands, feet, or any other part of the body to ascend a steep object. It is done for locomotion, recreation and competition, in trades that rely on it, and in emergency rescue and military operations. It is done indoors and out, on natural and man-made structures."
context = "Prince of the United Kingdom of Great Britain and Northern Ireland is a royal title normally granted to sons and grandsons of reigning and past British monarchs. It is also held by the Duke of Edinburgh, husband and consort of Queen Elizabeth II. The title is granted by the reigning monarch, who is the fount of all honours, through the issuing of letters patent as an expression of the royal will."

input_sentence = "join us at our gym we hope to see you there"

In [16]:
ranked_substitutions, topic_words_considered = generate_possible_pun_substitutions(context, input_sentence)

/home/wmvoje/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [17]:
for thing in [pun.substitute_pun(input_sentence, x[:3]) for x in ranked_substitutions[:10]]:
    print(thing)

join us at flower gym we hope to see you there
join us at our gym we hope to see use there
join us at our gym we hope to see you wear
join us at our gym we hope to sir you there
join us at our gym see hope to see you there
join us at our gym way hope to see you there
join use at our gym we hope to see you there
join us at our gym queen hope to see you there
join sir at our gym we hope to see you there
join pass at our gym we hope to see you there


In [18]:
ranked_substitutions

[['flower', 3, 2, 'doc2vec', 1, 0.7421815395355225, 2.6947584835533025],
 ['use', 9, 2, 'doc2vec', 1, 0.7421815395355225, 2.6947584835533025],
 ['wear', 10, 2, 'doc2vec', 1, 0.7421815395355225, 2.6947584835533025],
 ['sir', 8, 2, 'doc2vec', 2, 0.7250586152076721, 2.758397677168704],
 ['see', 5, 2, 'doc2vec', 3, 0.7234101295471191, 2.764683432414849],
 ['way', 5, 2, 'doc2vec', 3, 0.7234101295471191, 2.764683432414849],
 ['use', 1, 3, 'doc2vec', 1, 0.7421815395355225, 4.042137725329954],
 ['queen', 5, 3, 'doc2vec', 1, 0.7421815395355225, 4.042137725329954],
 ['sir', 1, 3, 'doc2vec', 2, 0.7250586152076721, 4.1375965157530565],
 ['pass', 1, 3, 'doc2vec', 2, 0.7250586152076721, 4.1375965157530565],
 ['man', 2, 3, 'doc2vec', 2, 0.7250586152076721, 4.1375965157530565],
 ['pass', 2, 3, 'doc2vec', 2, 0.7250586152076721, 4.1375965157530565],
 ['knight', 2, 3, 'doc2vec', 2, 0.7250586152076721, 4.1375965157530565],
 ['see', 1, 3, 'doc2vec', 3, 0.7234101295471191, 4.147025148622274],
 ['pass', 1, 3

In [19]:
topic_words_considered

[['prize', 'doc2vec', 1, 0.7421815395355225],
 ['use', 'doc2vec', 1, 0.7421815395355225],
 ['monarchy', 'doc2vec', 1, 0.7421815395355225],
 ['head', 'doc2vec', 1, 0.7421815395355225],
 ['chain', 'doc2vec', 1, 0.7421815395355225],
 ['choose', 'doc2vec', 1, 0.7421815395355225],
 ['receive', 'doc2vec', 1, 0.7421815395355225],
 ['crown', 'doc2vec', 1, 0.7421815395355225],
 ['queen', 'doc2vec', 1, 0.7421815395355225],
 ['win', 'doc2vec', 1, 0.7421815395355225],
 ['example', 'doc2vec', 1, 0.7421815395355225],
 ['couple', 'doc2vec', 1, 0.7421815395355225],
 ['monarch', 'doc2vec', 1, 0.7421815395355225],
 ['girl', 'doc2vec', 1, 0.7421815395355225],
 ['winners', 'doc2vec', 1, 0.7421815395355225],
 ['flower', 'doc2vec', 1, 0.7421815395355225],
 ['dance', 'doc2vec', 1, 0.7421815395355225],
 ['contest', 'doc2vec', 1, 0.7421815395355225],
 ['pageant', 'doc2vec', 1, 0.7421815395355225],
 ['occasion', 'doc2vec', 1, 0.7421815395355225],
 ['call', 'doc2vec', 1, 0.7421815395355225],
 ['headdress', 'doc2

In [24]:
def convert_topic_words_to_print(topic_words):
    output = [[]]
    topic_number = 1
    for topic in topic_words:
        if topic[1] == 'topicModel':
            return output
        if topic[2] != topic_number:
            topic_number = topic[2]
            output.append([topic[0]])
        output[topic_number-1].append(topic[0])
    
    return output

In [25]:
convert_topic_words_to_print(topic_words_considered)

[['prize',
  'use',
  'monarchy',
  'head',
  'chain',
  'choose',
  'receive',
  'crown',
  'queen',
  'win',
  'example',
  'couple',
  'monarch',
  'girl',
  'winners',
  'flower',
  'dance',
  'contest',
  'pageant',
  'occasion',
  'call',
  'headdress',
  'nothing',
  'symbol',
  'king',
  'wear',
  'part',
  'award',
  'type',
  'daisy',
  'beauty',
  'children'],
 ['chivalry',
  'chivalry',
  'title',
  'rank',
  'members',
  'baronets',
  'sir',
  'barons',
  'kingdom',
  'baronet',
  'order',
  'bachelor',
  'man',
  'pass',
  'children',
  'knight'],
 ['sister',
  'sister',
  'primogeniture',
  'inheritance',
  'monarchy',
  'see',
  'pass',
  'child',
  'cousin',
  'throne',
  'question',
  'oldest',
  'command',
  'law',
  'crown',
  'quit',
  'create',
  'beforehand',
  'monarch',
  'family',
  'time',
  'monarchies',
  'know',
  'monarchs',
  'idea',
  'system',
  'member',
  'past',
  'king',
  'arrange',
  'way',
  'nowadays',
  'methods',
  'die',
  'succession',
  'b

# Implementing TF-IDF

In [3]:
tokenized_corpus = [pun.tokenize(thing, stem=False, initial_word_split=False) for thing in wiki_doc2vec_corpus]

In [4]:
tokenized_corpus[0]

['month',
 'year',
 'come',
 'march',
 'months',
 'day',
 'begin',
 'day',
 'week',
 'years',
 'end',
 'day',
 'week',
 'flower',
 'daisy',
 'birthstone',
 'diamond',
 'mean',
 'diamond',
 'innocence']

In [56]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

In [74]:
dct = Dictionary(tokenized_corpus)  # fit dictionary


In [78]:
corpus = [dct.doc2bow(line) for line in tokenized_corpus]

In [79]:
model = TfidfModel(corpus)

In [80]:
model.num_docs

152497

In [86]:
dct.save('models/TF-IDF_dict.D')

In [81]:
test = model[corpus[2]]

In [89]:
import gensim

In [88]:
import pickle

gensim.utils.SaveLoad.save()
with open('models/TF-IDF_corpus.p', 'wb') as topick:
    pickle.dump(corpus, topick)

In [84]:
test = model[corpus[2]]

test.sort(key=lambda x: x[1])

for word, score in test:
    print(dct.id2token[word], score)

name 0.07642991265596688
come 0.09299847380709479
number 0.10308215631069341
end 0.10333525130180589
days 0.13522422413363386
august 0.1449337374796481
emperor 0.16665567988985144
begin 0.19614152904365406
calendar 0.20515363602679768
year 0.28992506519039224
day 0.4491260326768479
week 0.5080217645940343
month 0.5222296322234486


In [92]:
test

[(32, 0.07642991265596688),
 (2, 0.09299847380709479),
 (50, 0.10308215631069341),
 (6, 0.10333525130180589),
 (21, 0.13522422413363386),
 (48, 0.1449337374796481),
 (49, 0.16665567988985144),
 (0, 0.19614152904365406),
 (17, 0.20515363602679768),
 (14, 0.28992506519039224),
 (4, 0.4491260326768479),
 (13, 0.5080217645940343),
 (11, 0.5222296322234486)]

In [39]:
test = model[corp]

test.sort(key=lambda x: x[1])

for word, score in test:
    print(dct.id2token[word], score)

build 0.12450906492301654
church 0.16052042249390314
road 0.18368004313367003
market 0.19754512702055013
square 0.20926182074280714
edge 0.218723944519035
ruin 0.25152988694747286
hall 0.36196445127489196
pannier 0.507616944545011
town 0.5851299576409561


In [91]:
model.save('models/TF-IDF_model')

In [21]:
dct.doc2bow(tokenized_corpus[0])

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 3),
 (5, 2),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 2),
 (14, 1),
 (15, 1)]

In [23]:
dct.id2token[0]

'begin'

In [117]:
def tf_idf_of_document(document_id, model, dictionary, corpus, tf_idf_cutoff=0.07):
    
    # get the tf idf from the document id
    tf_idf = model[corpus[document_id]]
#     tf_idf.sort(key=lambda x: x[1])
    
    output = []
    tf_idf_value = None
    for index, tf_idf_value in tf_idf:
        if tf_idf_value > tf_idf_cutoff:
            output.append((dictionary.id2token[index], tf_idf_value))
    
    return output
    

In [118]:
tf_idf_of_document(1, model, dct, corpus)

[('come', 0.10673392377531137),
 ('day', 0.28994619210772093),
 ('march', 0.10383883276298947),
 ('month', 0.34962699596013763),
 ('week', 0.4372906947713977),
 ('year', 0.5545761226284901),
 ('years', 0.33699147777144156),
 ('celebrate', 0.15798987865350744),
 ('culture', 0.08404457453076937),
 ('days', 0.07759811231522494),
 ('fall', 0.0803843363913717),
 ('finish', 0.13069831455488717),
 ('follow', 0.0960991206553891),
 ('quite', 0.08402694480854042),
 ('spring', 0.09459211153512359),
 ('start', 0.10380382878112404),
 ('theory', 0.09507702380831191)]

In [ ]:
list_of

In [16]:
list_of_words = [word[0] for word in topic_words_considered]

In [25]:
pun.pos_tag(['choose'])

[('choose', 'NN')]

1. right topic 
2. Does it sound similar 

implement live validation *********

Conversational corpus 

multi-arm banding 


